# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,20.38,62,1,2.67,RO,1691037591
1,1,tiksi,71.6872,128.8694,5.54,89,100,6.28,RU,1691037721
2,2,edinburgh of the seven seas,-37.0676,-12.3116,13.91,83,96,17.53,SH,1691037721
3,3,yellowknife,62.4560,-114.3525,18.29,59,75,3.09,CA,1691037661
4,4,port lincoln,-34.7333,135.8667,22.04,43,80,10.40,AU,1691037722


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = .5,
    color = "City",
)

# Display the map plot
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = (city_data_df.loc[(city_data_df["Max Temp"]<28) & (city_data_df["Max Temp"]>22) &
                                (city_data_df["Humidity"]<75) & (city_data_df["Humidity"]>55) &
                                (city_data_df["Cloudiness"]<8)])

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,belyy yar,53.6039,91.3903,26.94,57,0,1.00,RU,1691037727
35,35,ypsonas,34.6884,32.9574,26.54,74,0,1.03,CY,1691037728
63,63,an nasr,31.2814,34.3025,26.75,65,0,1.74,PS,1691037734
134,134,papao,-17.7500,-149.5500,26.82,64,4,5.06,PF,1691037575
235,235,laguna,38.4210,-121.4238,24.19,61,0,4.63,US,1691037772
260,260,newman,37.3138,-121.0208,25.19,60,2,4.00,US,1691037777
312,312,misratah,32.3754,15.0925,26.51,70,0,2.23,LY,1691037788
322,322,pimentel,-6.8367,-79.9342,22.16,73,0,6.69,PE,1691037790
390,390,afaahiti,-17.7500,-149.2833,25.93,69,4,4.50,PF,1691037807
440,440,darnah,32.7670,22.6367,26.73,58,0,1.65,LY,1691037817


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
29,belyy yar,RU,53.6039,91.3903,57,
35,ypsonas,CY,34.6884,32.9574,74,
63,an nasr,PS,31.2814,34.3025,65,
134,papao,PF,-17.7500,-149.5500,64,
235,laguna,US,38.4210,-121.4238,61,
260,newman,US,37.3138,-121.0208,60,
312,misratah,LY,32.3754,15.0925,70,
322,pimentel,PE,-6.8367,-79.9342,73,
390,afaahiti,PF,-17.7500,-149.2833,69,
440,darnah,LY,32.7670,22.6367,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
belyy yar - nearest hotel: No hotel found
ypsonas - nearest hotel: Club Aphrodite
an nasr - nearest hotel: صالة قصر اللوتس
papao - nearest hotel: Hiti Moana Villa Lodge
laguna - nearest hotel: Holiday Inn Express & Suites
newman - nearest hotel: No hotel found
misratah - nearest hotel: الفندق السياحى
pimentel - nearest hotel: Hospedaje Calle del Sol
afaahiti - nearest hotel: Omati Lodge
darnah - nearest hotel: فندق الفردوس
bilbays - nearest hotel: No hotel found
akjoujt - nearest hotel: فندق فاطمة
ostrogozhsk - nearest hotel: No hotel found
harsova - nearest hotel: No hotel found
kill devil hills - nearest hotel: Mariner Days Inn & Suites
falconara marittima - nearest hotel: Hotel Touring


,City,Country,Lat,Lng,Humidity,Hotel Name
29,belyy yar,RU,53.6039,91.3903,57,No hotel found
35,ypsonas,CY,34.6884,32.9574,74,Club Aphrodite
63,an nasr,PS,31.2814,34.3025,65,صالة قصر اللوتس
134,papao,PF,-17.7500,-149.5500,64,Hiti Moana Villa Lodge
235,laguna,US,38.4210,-121.4238,61,Holiday Inn Express & Suites
260,newman,US,37.3138,-121.0208,60,No hotel found
312,misratah,LY,32.3754,15.0925,70,الفندق السياحى
322,pimentel,PE,-6.8367,-79.9342,73,Hospedaje Calle del Sol
390,afaahiti,PF,-17.7500,-149.2833,69,Omati Lodge
440,darnah,LY,32.7670,22.6367,58,فندق الفردوس


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)